# Downstream Classification

In [1]:
import os
import os.path
import sys
from IPython.display import clear_output
from time import sleep

import numpy as np
import matplotlib.pyplot as plt
from random import randrange


import torch
import torch.nn.functional as F
from torchvision import transforms, datasets, models
from torch import nn, optim
from torch.utils.data.sampler import Sampler

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [2]:
def test_data_loader(data_loader):
    anchors, positives, labels = iter(data_loader).next()
    print(anchors.shape)
    anchors = anchors.permute(0,2,3,1)
    positives = positives.permute(0,2,3,1)
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(nrows=2, ncols=2)
    print(dataset.classes)
    print(labels[0])
    ax1.imshow(anchors[0].squeeze(-1))
    ax2.imshow(positives[0].squeeze(-1))
    print(labels[1])
    ax3.imshow(anchors[1].squeeze(-1))
    ax4.imshow(positives[1].squeeze(-1))

### MNIST

In [3]:
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.1307,), (0.3081,)),
                              ])
# Download and load the training data
trainset = datasets.MNIST('data', download=True, train=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

testset = datasets.MNIST('data', download=True, train=False, transform=transform)
testloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

In [4]:
from model.mnist_model import MNISTNet

z_dim = 3
model = MNISTNet(z_dim = z_dim)

file_name = '{}_{}d.pth'.format(type(model).__name__, z_dim)
PATH = './pth_files/{}'.format(file_name)
state_dict = torch.load(PATH)
model.load_state_dict(state_dict)

for param in model.parameters():
    param.requires_grad = False
    
model.g = nn.Linear(10, 10)
model.to(device)
print(model)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.g.parameters(), lr=0.001)

MNISTNet(
  (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(20, 50, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=800, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=10, bias=True)
  (g): Linear(in_features=10, out_features=10, bias=True)
)


In [5]:
epochs = 5

train_losses, test_losses = [], []
for e in range(epochs):
    running_loss = 0
    for images, labels in trainloader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        
        log_ps = model(images)
        loss = criterion(log_ps, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
    else:
        test_loss = 0
        accuracy = 0
        
        # Turn off gradients for validation, saves memory and computations
        with torch.no_grad():
            model.eval()
            for images, labels in testloader:
                images, labels = images.to(device), labels.to(device)
                log_ps = model(images)
                test_loss += criterion(log_ps, labels)
                
                ps = torch.exp(log_ps)
                top_p, top_class = ps.topk(1, dim=1)
                equals = top_class == labels.view(*top_class.shape)
                accuracy += torch.mean(equals.type(torch.FloatTensor))
        
        model.train()
        
        train_losses.append(running_loss/len(trainloader))
        test_losses.append(test_loss/len(testloader))

        print("Epoch: {}/{}.. ".format(e+1, epochs),
              "Training Loss: {:.3f}.. ".format(train_losses[-1]),
              "Test Loss: {:.3f}.. ".format(test_losses[-1]),
              "Test Accuracy: {:.3f}".format(accuracy/len(testloader)))

Epoch: 1/5..  Training Loss: 1.423..  Test Loss: 0.749..  Test Accuracy: 0.953
Epoch: 2/5..  Training Loss: 0.565..  Test Loss: 0.436..  Test Accuracy: 0.957
Epoch: 3/5..  Training Loss: 0.364..  Test Loss: 0.308..  Test Accuracy: 0.958
Epoch: 4/5..  Training Loss: 0.275..  Test Loss: 0.247..  Test Accuracy: 0.958
Epoch: 5/5..  Training Loss: 0.228..  Test Loss: 0.213..  Test Accuracy: 0.960


### CIFAR

In [ ]:
transform = transforms.Compose([
    #transforms.RandomHorizontalFlip(), # randomly flip and rotate
    #transforms.RandomRotation(10),
    transforms.ToTensor(),
    #transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

trainset = datasets.CIFAR10(root='data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32,
                                          shuffle=True, num_workers=4)

testdata = datasets.CIFAR10('data', train=False,
                             download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testdata, batch_size=32, 
    num_workers=4)

In [ ]:
from model.cifar_model import CIFARNet2

z_dim = 50
model = CIFARNet2(z_dim = z_dim)

#file_name = '{}_{}d.pth'.format(type(model).__name__, z_dim)
#PATH = './pth_files/{}'.format(file_name)
#state_dict = torch.load(PATH)
#model.load_state_dict(state_dict)

# Decide whether to freeze or finetune the model
for param in model.parameters():
    param.requires_grad = False

model.g = nn.Sequential(
    nn.Linear(500, 100),
    nn.ReLU(),
    nn.Linear(100, 10)
)

model.to(device)
print(model)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
epochs = 10

train_losses, test_losses = [], []
for e in range(epochs):
    running_loss = 0
    for images, labels in trainloader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        
        log_ps = model(images)
        loss = criterion(log_ps, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
    else:
        test_loss = 0
        accuracy = 0
        
        # Turn off gradients for validation, saves memory and computations
        with torch.no_grad():
            model.eval()
            for images, labels in testloader:
                images, labels = images.to(device), labels.to(device)
                log_ps = model(images)
                test_loss += criterion(log_ps, labels)
                
                ps = torch.exp(log_ps)
                top_p, top_class = ps.topk(1, dim=1)
                equals = top_class == labels.view(*top_class.shape)
                accuracy += torch.mean(equals.type(torch.FloatTensor))
        
        model.train()
        
        train_losses.append(running_loss/len(trainloader))
        test_losses.append(test_loss/len(testloader))

        print("Epoch: {}/{}.. ".format(e+1, epochs),
              "Training Loss: {:.3f}.. ".format(train_losses[-1]),
              "Test Loss: {:.3f}.. ".format(test_losses[-1]),
              "Test Accuracy: {:.3f}".format(accuracy/len(testloader)))